In [1]:
from google.colab import drive
import os
import gdown # Library to download files from Google Drive
!gdown 1JdGaXmUnItfCn8ucn_5kBfDeh3NzrmMs # Google Drive ID of the zip file to be downloaded


Downloading...
From (original): https://drive.google.com/uc?id=1JdGaXmUnItfCn8ucn_5kBfDeh3NzrmMs
From (redirected): https://drive.google.com/uc?id=1JdGaXmUnItfCn8ucn_5kBfDeh3NzrmMs&confirm=t&uuid=8cd1ab7e-7fee-4c6c-9bcf-bc9eab262224
To: /content/images.zip adlı dosyanın kopyası
100% 128M/128M [00:00<00:00, 133MB/s]


In [3]:
!unzip -oq images # Unzip the file downloaded. Options -o and -q overwrites the files if exists already and disables printing out the extracted files, respectively.

In [4]:

root_dir = '/content/images'


In [18]:
from PIL import Image, UnidentifiedImageError
import os

root_dir = '/content/images'
output_dir = '/content/cropped_images'

# Function to calculate average resolution of images in a directory
def calculate_average_resolution(directory):
    total_width = 0
    total_height = 0
    num_images = 0

    # Loop through each file in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        try:
            # Check if the file is an image
            if os.path.isfile(filepath) and any(filename.lower().endswith(ext) for ext in ['.png', '.jpg', '.jpeg']):
                # Open the image and get its dimensions
                with Image.open(filepath) as img:
                    width, height = img.size
                    total_width += width
                    total_height += height
                    num_images += 1
        except (UnidentifiedImageError, OSError):
            print(f"Unable to process image file: {filename}")
            continue

    if num_images == 0:
        print("No images found in the directory.")
        return None
    else:
        avg_width = total_width // num_images
        avg_height = total_height // num_images
        return (avg_width, avg_height, num_images)

# Function to create the output directory if it doesn't exist
def create_output_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to crop images in a directory to a specified resolution
def crop_images_to_resolution(directory, target_resolution):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        try:
            # Check if the file is an image
            if os.path.isfile(filepath) and any(filename.lower().endswith(ext) for ext in ['.png', '.jpg', '.jpeg']):
                # Open the image and crop it to the target resolution
                with Image.open(filepath) as img:
                    img = img.crop((0, 0, target_resolution[0], target_resolution[1]))
                    img.save(os.path.join(output_dir, filename))
        except (UnidentifiedImageError, OSError):
            print(f"Unable to process image file: {filename}")
            continue

# Calculate average resolution and total number of images
result = calculate_average_resolution(root_dir)

if result:
    avg_resolution, num_images = result[:2], result[2]
    print(f"Total number of images: {num_images}")
    print(f"Average resolution: {avg_resolution[0]}x{avg_resolution[1]} pixels")

    # Create the output directory if it doesn't exist
    create_output_directory(output_dir)

    # Crop images to average resolution
    crop_images_to_resolution(root_dir, avg_resolution)
    print("Images cropped to average resolution.")
else:
    print("Unable to calculate average resolution.")


Unable to process image file: 695766190760630.jpg
Total number of images: 635
Average resolution: 1193x869 pixels
Unable to process image file: 695766190760630.jpg
Unable to process image file: 1969722553066031.jpg
Unable to process image file: 176574166615555.jpg
Unable to process image file: 728334204189811.jpg
Unable to process image file: 430743550786558.jpg
Unable to process image file: 1212019935620496.jpg
Unable to process image file: 676687789384754.jpg
Unable to process image file: 320619618717364.jpg
Unable to process image file: 2167198373551810.jpg
Unable to process image file: 1684090145051513.jpg
Unable to process image file: 253935178579446.jpg
Unable to process image file: 309519166320626.jpg
Unable to process image file: 499615317172296.jpg
Unable to process image file: 186076198984051.jpg
Unable to process image file: 1723492444426167.jpg
Unable to process image file: 285424975595779.jpg
Unable to process image file: 346700996063593.jpg
Unable to process image file: 9

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate

# Image input branch
image_input = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)

# Font type input branch
font_input = Input(shape=(num_font_types,))
# Color palette input branch
color_input = Input(shape=(num_colors,))

# Concatenate all features
concatenated = Concatenate()([x, font_input, color_input])

# Dense layers for prediction
y = Dense(128, activation='relu')(concatenated)
y = Dense(64, activation='relu')(y)
output = Dense(1)(y)  # CPI output

model = Model(inputs=[image_input, font_input, color_input], outputs=output)
model.compile(optimizer='adam', loss='mean_absolute_error')

model.summary()